In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:1")
messages = [{"role": "user","content": "test!"}]
empty_magpie_template = tokenizer.apply_chat_template(messages, tokenize=False)
empty_magpie_template